In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive
import numpy as np

drive.mount('/content/drive')
dffile = '/content/drive/MyDrive/vgames2.csv'

Mounted at /content/drive


In [ ]:
df = pd.read_csv(dffile, index_col = 0)
df.head()

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
1,Candace Kane's Candy Factory,DS,2008.0,Action,Destineer,0.04,0,0,0
2,The Munchables,Wii,2009.0,Action,Namco Bandai Games,0.17,0,0,0.01
3,Otome wa Oanesama Boku ni Koi Shiteru Portable,PSP,2010.0,Adventure,Alchemist,0,0,0.02,0
4,Deal or No Deal: Special Edition,DS,2010.0,Misc,Zoo Games,0.04,0,0,0
5,Ben 10 Ultimate Alien: Cosmic Destruction,PS3,2010.0,Platform,D3Publisher,0.12,0.09,0,0.04


#[ 매출량을 예측해보는 모델 제작 ]

In [ ]:
# 판매량 float형으로 변경
df_sales = df[['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales']]
df_sales2 = pd.DataFrame(index = df_sales.index-1, columns = df_sales.columns)
# list별로 처리해주기 위해 임시로 빈 리스트 선언
temp = []
for i in range(len(df_sales)):
  temp = df_sales.iloc[i]
  for j in range(len(temp)):
    if 'K' in temp[j]:
      temp[j] = temp[j].replace('K', '')
      temp[j] = float(temp[j]) * 0.001
    elif 'M' in temp[j]:
      temp[j] = temp[j].replace('M', '')
  df_sales2.loc[i] = temp

df_sales2 = df_sales2.astype(float)
df_sales2

,NA_Sales,EU_Sales,JP_Sales,Other_Sales
0,0.04,0.00,0.00,0.00
1,0.17,0.00,0.00,0.01
2,0.00,0.00,0.02,0.00
3,0.04,0.00,0.00,0.00
4,0.12,0.09,0.00,0.04
...,...,...,...,...
16593,0.15,0.04,0.00,0.01
16594,0.01,0.00,0.00,0.00
16595,0.44,0.19,0.03,0.13
16596,0.05,0.05,0.25,0.03


In [ ]:
# 총 판매량 추가
df_sales2['Total_Sales'] = df_sales2.sum(axis = 1)
df_sales2.index = df_sales2.index
df_sales2

,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Total_Sales
0,0.04,0.00,0.00,0.00,0.04
1,0.17,0.00,0.00,0.01,0.18
2,0.00,0.00,0.02,0.00,0.02
3,0.04,0.00,0.00,0.00,0.04
4,0.12,0.09,0.00,0.04,0.25
...,...,...,...,...,...
16593,0.15,0.04,0.00,0.01,0.20
16594,0.01,0.00,0.00,0.00,0.01
16595,0.44,0.19,0.03,0.13,0.79
16596,0.05,0.05,0.25,0.03,0.38


In [ ]:
df = df.reset_index()

game_df = pd.concat([df[['Name', 'Platform', 'Year', 'Genre', 'Publisher']], df_sales2], axis=1)
game_df.head()

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Total_Sales
0,Candace Kane's Candy Factory,DS,2008.0,Action,Destineer,0.04,0.00,0.00,0.00,0.04
1,The Munchables,Wii,2009.0,Action,Namco Bandai Games,0.17,0.00,0.00,0.01,0.18
2,Otome wa Oanesama Boku ni Koi Shiteru Portable,PSP,2010.0,Adventure,Alchemist,0.00,0.00,0.02,0.00,0.02
3,Deal or No Deal: Special Edition,DS,2010.0,Misc,Zoo Games,0.04,0.00,0.00,0.00,0.04
4,Ben 10 Ultimate Alien: Cosmic Destruction,PS3,2010.0,Platform,D3Publisher,0.12,0.09,0.00,0.04,0.25


In [ ]:
game_df.isnull().sum()

Name             0
Platform         0
Year           271
Genre           50
Publisher       58
NA_Sales         0
EU_Sales         0
JP_Sales         0
Other_Sales      0
Total_Sales      0
dtype: int64

In [ ]:
game_df = game_df.dropna()

In [ ]:
# 총 매출량을 예측할 것이므로 다른 지역에서의 매출량을 제외
# 이름은 예측에 도움이 되지 않으므로 제외
game_df2 = game_df[['Platform', 'Year', 'Genre', 'Total_Sales']]

# [ 모델 ]


In [ ]:
target = 'Total_Sales'

X_train = game_df2.drop(columns=target)

y_train = game_df2[target]

In [ ]:
# pip install category_encoders

     |████████████████████████████████| 82 kB 415 kB/s 


In [ ]:
# 매출량을 예측하는 모델이므로 회귀모델을 사용
from sklearn.pipeline import make_pipeline
from category_encoders import OrdinalEncoder
from sklearn.ensemble import RandomForestRegressor

pipe = make_pipeline(
    OrdinalEncoder(),
    RandomForestRegressor(n_estimators=300, max_depth=5, n_jobs=-1, random_state=10)
)

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
pipe.fit(X_train, y_train);